In [3]:
import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
import os
from glob2 import glob
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [4]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [35]:
train_dir = 'dataset/nomal_data/train'
test_dir = 'dataset/nomal_data/test'
an_dir = 'data2/anomal_data/'
image_size = (128, 128)
batch_size = 32

In [36]:
train_datagen = ImageDataGenerator(rescale=1. / 255)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_gen = train_datagen.flow_from_directory(train_dir,
                                              target_size=image_size,
                                              batch_size=batch_size,
                                              color_mode='grayscale',
                                              class_mode='input')

test_gen = test_datagen.flow_from_directory(test_dir,
                                            target_size=image_size,
                                            batch_size=batch_size,
                                            color_mode='grayscale',
                                            class_mode='input')

an_gen = test_datagen.flow_from_directory(an_dir,
                                          target_size=image_size,
                                          batch_size=batch_size,
                                          color_mode='grayscale',
                                          class_mode='input')

Found 8064 images belonging to 1 classes.
Found 1008 images belonging to 1 classes.
Found 322 images belonging to 1 classes.


In [37]:
class Autoencoder(Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = tf.keras.Sequential([
            layers.Conv2D(8, 3, activation='relu'),
            layers.MaxPool2D(),
            layers.Conv2D(16, 3, activation='relu'),
            layers.MaxPool2D(),
            layers.Conv2D(32, 3, activation='relu')
        ])
        
        self.decoder = tf.keras.Sequential([
            layers.UpSampling2D(),
            layers.Conv2DTranspose(16, 3, activation='relu'),
            layers.UpSampling2D(),
            layers.Conv2DTranspose(8, 3, activation='relu'),
            layers.Conv2DTranspose(4, 3, activation='relu'),
            layers.Conv2D(1, 3, activation='sigmoid', padding='same')
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded


xs, ys = next(train_gen)
autoencoder = Autoencoder()
autoencoder.build(xs.shape)
autoencoder.encoder.summary()
autoencoder.decoder.summary()

Model: "sequential_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_94 (Conv2D)           (32, 126, 126, 8)         80        
_________________________________________________________________
max_pooling2d_45 (MaxPooling (32, 63, 63, 8)           0         
_________________________________________________________________
conv2d_95 (Conv2D)           (32, 61, 61, 16)          1168      
_________________________________________________________________
max_pooling2d_46 (MaxPooling (32, 30, 30, 16)          0         
_________________________________________________________________
conv2d_96 (Conv2D)           (32, 28, 28, 32)          4640      
Total params: 5,888
Trainable params: 5,888
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_51"
_________________________________________________________________
Layer (type)                

In [38]:
autoencoder = Autoencoder()
autoencoder.compile(optimizer='adam', loss=losses.MeanSquaredError())

checkpoint_path = "./checkpoint/cp.ckpt"

# autoencoder.load_weights(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

autoencoder.fit(train_gen,
                epochs=5,
                shuffle=True,
                validation_data=test_gen,
                callbacks=[cp_callback])

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]

In [ ]:
autoencoder.load_weights(checkpoint_path)

In [ ]:
xs, ys = next(train_gen)
encoded_imgs = autoencoder.encoder(xs).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(xs[i])
    plt.title("original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i])
    plt.title("reconstructed")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
def cal_loss(gen):
    xs, ys = next(gen)
    encoded_imgs = autoencoder.encoder(xs).numpy()
    recs = autoencoder.decoder(encoded_imgs).numpy()
    mse = [(np.square(rec - img)).mean() for rec, img in zip(recs, xs)]
    return mse


mse = cal_loss(train_gen)
print(mse)
plt.hist(mse, bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()

mse2 = cal_loss(an_gen)
print(mse2)
plt.hist(mse2, bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()

In [ ]:
xs, ys = next(an_gen)
encoded_imgs = autoencoder.encoder(xs).numpy()
decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(xs[i])
    plt.title("original")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i])
    plt.title("reconstructed")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()